(Old)
I think I'm observing that patching all paths to a node leads to different results than pathching all paths from a node 
(this is happening in the context of the circuit discovery hypothesis testing, where I ablate all edges that aren't in paths)
    # note - maybe I should experiment with only ablating not reachable edges vs edges that don't lead to dest, maybe its only one side
    # yes, ok, its just the reachable ablations that are causing issues

(New)
So the problem is E[f(x)] != f(E[x]). Ih the case of mean ablation, the result of all the inputs being mean ablated is not the mean, its something else. Resample ablation solves this by ablating towards a single input, and low and behold we get the exact same answer

so, to test this, I want patch all input edges to a node, and compare performance with pathching all outgoing egdes from the node

is this a function how the masking works? 

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5" #"1"
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [2]:
from typing import Callable, Dict, Tuple, Union, Optional, Any, Literal, NamedTuple
from itertools import product
from copy import deepcopy
import random
import math
from collections import defaultdict

import torch
import numpy as np
from scipy.stats import binom, beta

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

from transformer_lens import HookedTransformer

from auto_circuit.types import (
    CircuitOutputs, 
    BatchKey,
    PruneScores,
    PatchType, 
    AblationType,
    SrcNode, 
    DestNode, 
    Edge,
    Node
)
from auto_circuit.data import PromptPairBatch, PromptDataLoader   
from auto_circuit.prune_algos.mask_gradient import mask_gradient_prune_scores
from auto_circuit.tasks import (
    Task,
    DOCSTRING_COMPONENT_CIRCUIT_TASK, 
    DOCSTRING_TOKEN_CIRCUIT_TASK,
    IOI_COMPONENT_CIRCUIT_TASK, 
    IOI_TOKEN_CIRCUIT_TASK,
    IOI_GPT2_AUTOENCODER_COMPONENT_CIRCUIT_TASK,
    GREATERTHAN_COMPONENT_CIRCUIT_TASK,  
    GREATERTHAN_GPT2_AUTOENCODER_COMPONENT_CIRCUIT_TASK, 
    CAPITAL_CITIES_PYTHIA_70M_AUTOENCODER_COMPONENT_CIRCUIT_TASK, 
    TRACR_REVERSE_TOKEN_CIRCUIT_TASK,
    docstring_true_edges
)
from auto_circuit.visualize import draw_seq_graph
from auto_circuit.utils.tensor_ops import batch_answer_diffs, batch_avg_answer_val
from auto_circuit.utils.patchable_model import PatchableModel
from auto_circuit.utils.custom_tqdm import tqdm

from elk_experiments.auto_circuit.auto_circuit_utils import (
    run_circuits,
    desc_prune_scores, 
    prune_scores_threshold, 
    load_tf_model
)
from elk_experiments.auto_circuit.score_funcs import GradFunc, AnswerFunc, get_score_func

from elk_experiments.auto_circuit.circuit_hypotests import (
    get_edge_idx, 
    edges_from_mask,
    equiv_test,
    sweep_search_smallest_equiv,
    plot_num_ablated_C_gt_M, 
    plot_circuit_and_model_scores,
    compute_knees, 
    plot_edge_scores_and_knees,
    minimality_test, 
    plot_p_values, 
    plot_edge_k, 
    plot_score_quantiles,
    independence_test
)

from elk_experiments.auto_circuit.node_graph import (
    NodeGraph, 
    SeqNode, 
    NodeIdx,
    SampleType,
    get_node_idx, 
    sample_paths, 
    visualize_graph, 
    edge_in_path
)

In [7]:
import random 
from auto_circuit.utils.ablation_activations import src_ablations, batch_src_ablations
from auto_circuit.utils.graph_utils import patch_mode
torch.set_grad_enabled(False)

In [4]:
# load docstring 
task = DOCSTRING_TOKEN_CIRCUIT_TASK
task.init_task()

Loaded pretrained model attn-only-4l into HookedTransformer
seq_len before divergence 41
seq_len after divergence 30


In [8]:
# ok, so lets take an edge in the offical circuit
edges = [next(edge for edge in task.true_edges if edge.src.layer == 2)]#random.sample(list(task.true_edges), 9)

# run the ablations 
print("computing ablations")
ablations = batch_src_ablations(task.model, task.train_loader, AblationType.RESAMPLE, "corrupt")

computing ablations


In [9]:
edges[0].dest.module_name, edges[0].seq_idx, edges[0].src.src_idx

('blocks.3.hook_resid_post', 11, 9)

In [10]:
# run the model patching all inputs to the edge
print("running model on patched inputs")
input_edges = []
for model_edge in task.model.edges:
    for edge in edges:
        if not get_node_idx(model_edge.dest) == get_node_idx(edge.src):
            continue
        # if egge is a k or q 
        if edge.src.name.startswith("A") and model_edge.dest.name.endswith(("K", "V")):
            input_edges.append(model_edge)
        elif edge.seq_idx == model_edge.seq_idx:
            input_edges.append(model_edge)

running model on patched inputs


In [23]:
# so how to check that we're fully patching the src node? 

# the input to the src node should be the same as on the fully ablated examples
edges[0].src.src_idx, edges[0].dest.module_name, edges[0].seq_idx
# so at this module and this seq idx, the patch src outs should equal the ablation src outs

(9, 'blocks.2.hook_v_input', 7)

In [11]:
# run with input edges ablated
patched_input_out = []
for batch in tqdm(task.train_loader):
    with patch_mode(task.model, ablations[batch.key], input_edges):
        patched_input_out.append(task.model(batch.clean)[task.model.out_slice])
patched_input_out = torch.cat(patched_input_out, dim=0)

# finally, run the model patching all the inputs to the edge and the edge itself
print('running model on patched inputs and edge')
patched_input_edge_output = []
for batch in tqdm(task.train_loader):
    with patch_mode(task.model, ablations[batch.key], input_edges + edges):
        patched_input_edge_output.append(task.model(batch.clean)[task.model.out_slice])
patched_input_edge_output = torch.cat(patched_input_edge_output, dim=0)


running model on patched inputs and edge


In [12]:
patched_input_out = torch.cat(patched_input_out, dim=0)
patched_input_edge_output = torch.cat(patched_input_edge_output, dim=0)

In [13]:
# compare the outputs 
print("comparing results")
diffs = patched_input_out - patched_input_edge_output
equal = torch.allclose(patched_input_out, patched_input_edge_output, atol=1e-4)
equal, diffs.topk(10)


comparing results


(True,
 torch.return_types.topk(
 values=tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'),
 indices=tensor([[7, 6, 4,  ..., 3, 8, 9],
         [7, 6, 4,  ..., 3, 8, 9],
         [7, 6, 4,  ..., 3, 8, 9],
         ...,
         [7, 6, 4,  ..., 3, 8, 9],
         [7, 6, 4,  ..., 3, 8, 9],
         [7, 6, 4,  ..., 3, 8, 9]], device='cuda:0')))